In [ ]:
import imaplib
import email
from email import policy
from email.utils import parsedate_to_datetime
import pytz
from io import StringIO
import csv
import os
import time

import gspread
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
from datetime import datetime
from gspread_dataframe import set_with_dataframe

from simple_salesforce import Salesforce, SalesforceLogin
from simple_salesforce.exceptions import SalesforceAuthenticationFailed, SalesforceGeneralError
import requests
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# --- Configuración ---
EMAIL = "yonatan.osorio@openenglish.com"
PASSWORD_EMAIL = "wfpm nswd umrn rssj"

ASUNTOS_A_BUSCAR = {
    #Leads
    "Report results (Lead OE Adults": "Lead OE Adults",
    
    
    #Dials
    "Scheduled Report: Dials Control Panel Adults": "Dials OE Adults",
    "Scheduled Report: Dials Control Panel Junior": "Dials OE Junior",
    "Scheduled Report: Dials Control Panel Turkey": "Dials OE Turkey",
    "Scheduled Report: Dials Control Panel BR": "Dials OE BR",
    "Scheduled Report: Dials Control Panel Espana": "Dials OE Espana",
    "Scheduled Report: Dials Control Panel Vzla": "Dials OE Vzla",
    
    #Login Time
    "Scheduled Report: WFM Control Panel Adults": "Login Time OE Adultos",
    "Scheduled Report: WFM Control Panel Junior": "Login Time OE Junior",
    "Scheduled Report: WFM Control Panel Turkey": "Login Time OE Turkey",
    "Scheduled Report: WFM Control Panel BR": "Login Time OE BR",
    "Scheduled Report: WFM Control Panel Espana": "Login Time OE Espana"
}

# Restricciones horarias por asunto (en horas, formato 24h)
RESTRICCIONES_HORARIAS = {
    "Scheduled Report: Dials Control Panel OE Vzla": 21,
    "Scheduled Report: Dials Control Panel OE Jrr": 21,
    #"Scheduled Report: WFM Control panel OE": 15,
    #"Report results (Lead OE Adults": 22,
    
}

DESCARGAR_DESPUES = {
    "Report results (Lead OE Adults": 17,
}

SHEET_MAPPINGS = {
    #Leads Adults (solo despues de las 17:00)
    "Report results (Lead OE Adults": {
        "spreadsheet_id": "1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4",
        "worksheet_name": "Leads"
    },
    #Dials
    "Scheduled Report: Dials Control Panel Adults": {
        "spreadsheet_id": "1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4",
        "worksheet_name": "Dials"
    },
    "Scheduled Report: Dials Control Panel Junior": {
        "spreadsheet_id": "1wXbaCBJrC_TxXgaGo-14NVAkS3uzRo_a6oWUP5ux7KU",
        "worksheet_name": "Dials"
    },
    "Scheduled Report: Dials Control Panel Turkey": {
        "spreadsheet_id": "1bA1GEhfufFq9_iLczSMzhFGFbKZoHwe7K4g_9g_vCsU",
        "worksheet_name": "Dials"
    },
    "Scheduled Report: Dials Control Panel BR": {
        "spreadsheet_id": "14t9cSHrJ1VexbOtm1ybmYPg9iiewuAK2j1YEPP4rB4E",
        "worksheet_name": "Dials"
    },
    "Scheduled Report: Dials Control Panel Espana": {
        "spreadsheet_id": "1LxXDXw8B4xP0Y6_PVexnaXpnBUUPL7kNP9IJiULBo6o",
        "worksheet_name": "Dials"
    },
    "Scheduled Report: Dials Control Panel Vzla": {
        "spreadsheet_id": "1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80",
        "worksheet_name": "Dials"
    },
    
    #Login Time
    "Scheduled Report: WFM Control Panel Adults": {
        "spreadsheet_id": "1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4",
        "worksheet_name": "Login Time",
        "range": "E1"
    },
    "Scheduled Report: WFM Control Panel Junior": {
        "spreadsheet_id": "1wXbaCBJrC_TxXgaGo-14NVAkS3uzRo_a6oWUP5ux7KU",
        "worksheet_name": "Login Time",
        "range": "E1"
    },
    "Scheduled Report: WFM Control Panel Turkey": {
        "spreadsheet_id": "1bA1GEhfufFq9_iLczSMzhFGFbKZoHwe7K4g_9g_vCsU",
        "worksheet_name": "Login Time",
        "range": "E1"
    },
    "Scheduled Report: WFM Control Panel BR": {
        "spreadsheet_id": "14t9cSHrJ1VexbOtm1ybmYPg9iiewuAK2j1YEPP4rB4E",
        "worksheet_name": "Login Time",
        "range": "E1"
    },
    "Scheduled Report: WFM Control Panel Espana": {
        "spreadsheet_id": "1LxXDXw8B4xP0Y6_PVexnaXpnBUUPL7kNP9IJiULBo6o",
        "worksheet_name": "Login Time",
        "range": "E1"
    },
    
}

archivos_sheets = {
    "Lead OE Adults.csv": {
        "sheet_id": "1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4",
        "hoja": "Leads"
    },
    "Lead OE Junior.csv": {
        "sheet_id": "1wXbaCBJrC_TxXgaGo-14NVAkS3uzRo_a6oWUP5ux7KU",
        "hoja": "Leads"
    },
    "Lead OE Turkey.csv": {
        "sheet_id": "1bA1GEhfufFq9_iLczSMzhFGFbKZoHwe7K4g_9g_vCsU",
        "hoja": "Leads"
    },
    "Lead OE Brasil.csv": {
        "sheet_id": "14t9cSHrJ1VexbOtm1ybmYPg9iiewuAK2j1YEPP4rB4E",
        "hoja": "Leads"
    },
    "Lead OE España.csv": {
        "sheet_id": "1LxXDXw8B4xP0Y6_PVexnaXpnBUUPL7kNP9IJiULBo6o",
        "hoja": "Leads"
    },
    "Lead OE Venezuela.csv": {
        "sheet_id": "1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80",
        "hoja": "Leads"
    },
    # Agrega más archivos aquí si lo necesitas
}
USERNAME = 'yonatan.osorio@openenglish.com'
PASSWORD = 'Yonatan26++'
SECURITY_TOKEN = 'ho7zyIcpDbybcP1G7tLbor6sU'  # si aplica
DOMAIN = 'login'  # usa 'test' para sandbox

# ID del reporte que quieres descargar
REPORTS = {
    'Lead OE Adults': '00O3r000007Y8AX',
    'Lead OE Junior': '00O3r000007Y8hH',
    'Lead OE España': '00OKV000008PVBz',
    'Lead OE Venezuela': '00O3r000007Y8bs',
    'Lead OE Turkey': '00O3r000007Y959',
    'Lead OE Brasil': '00O3r000007Y8dU'
}

omitidos_por_tiempo = []
# Lista para reportes omitidos por restricción de hora

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CREDENTIALS_FILE = 'panels-id.json'
TOKEN_FILE = 'token.json'

# --- Configuración carpeta destino ---
FOLDER_PATH = 'Descargas'
LOG_FOLDER = os.path.join(FOLDER_PATH, 'Logs')
os.makedirs(FOLDER_PATH, exist_ok=True)
os.makedirs(LOG_FOLDER, exist_ok=True)

def descargar_reportes_salesforce():
    # Configura tus credenciales aquí
    try:
        session_id, instance = SalesforceLogin(
            username=USERNAME,
            password=PASSWORD,
            security_token=SECURITY_TOKEN,
            domain=DOMAIN
        )
        sf = Salesforce(instance=instance, session_id=session_id)
        print("✅ Conexión exitosa a Salesforce")

    except SalesforceAuthenticationFailed as e:
        print(f"❌ Error de autenticación: {e}")
        return
    except Exception as e:
        print(f"❌ Error general al autenticar: {e}")
        return

    current_time = datetime.now().time()
    limit_time_adults = datetime.strptime("17:00", "%H:%M").time()
    limit_time_spain = datetime.strptime("21:00", "%H:%M").time()
    limit_time_turkey = datetime.strptime("18:00", "%H:%M").time()

    try:
        for report_name, report_id in REPORTS.items():
            log_path = os.path.join(LOG_FOLDER)
            omitido = False

            if report_name == 'Lead OE Adults' and current_time >= limit_time_adults:
                mensaje = f"❌ No se descargará '{report_name}' (Límite: {limit_time_adults})\n"
                omitido = True
            elif report_name == 'Lead OE España' and current_time >= limit_time_spain:
                mensaje = f"❌ No se descargará '{report_name}' (Límite: {limit_time_spain})\n"
                omitido = True
            elif report_name == 'Lead OE Turkey' and current_time >= limit_time_turkey:
                mensaje = f"❌ No se descargará '{report_name}' (Límite: {limit_time_turkey})\n"
                omitido = True

            if omitido:
                print(mensaje)
                omitidos_por_tiempo.append(f"{report_name}.csv")  # Guardar nombre de archivo omitido
                with open(os.path.join(log_path, f"{report_name}_omitido.txt"), 'a', encoding='utf-8') as f:
                    f.write(mensaje)
                    f.write(f"⏰ {datetime.now()}\n\n")
                continue

            print(f"\n📄 Procesando '{report_name}'...")
            report_url = f"https://{sf.sf_instance}/services/data/v{sf.sf_version}/analytics/reports/{report_id}"
            headers = {'Authorization': f'Bearer {sf.session_id}'}
            response = requests.get(report_url, headers=headers)
            response.raise_for_status()

            report_data = response.json()
            rows = report_data.get("factMap", {}).get("T!T", {}).get("rows", [])
            total_registros = len(rows)
            column_info = report_data.get("reportExtendedMetadata", {}).get("detailColumnInfo", {})
            header_labels = [info.get("label", key) for key, info in column_info.items()]

            filename = os.path.join(FOLDER_PATH, f"{report_name}.csv")
            with open(filename, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(header_labels)
                for row in rows:
                    data_row = [cell.get("label", "") for cell in row.get("dataCells", [])]
                    writer.writerow(data_row)

            print(f"✅ Guardado '{filename}' con {total_registros} registros")
            with open(os.path.join(log_path, f"{report_name}.txt"), 'a', encoding='utf-8') as f:
                f.write(f"✅ Reporte '{report_name}' descargado\n")
                f.write(f"📁 Archivo: {filename}\n")
                f.write(f"📊 Registros: {total_registros}\n")
                f.write(f"⏰ Fecha y hora: {datetime.now()}\n")

    except Exception as e:
        print(f"❌ Error durante la descarga de reportes: {e}")

    finally:
        try:
            if 'sf' in locals():
                logout_url = f"https://{sf.sf_instance}/services/oauth2/revoke"
                logout_response = requests.post(logout_url, data={'token': sf.session_id})
                if logout_response.status_code in (200, 204):
                    print("🔒 Sesión cerrada correctamente.")
                else:
                    print("⚠️ Error al cerrar sesión:", logout_response.text)
        except Exception as logout_err:
            print(f"⚠️ Error al cerrar sesión: {logout_err}")

    # Subir archivos manuales a Google Sheets
    try:
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
        service = build('sheets', 'v4', credentials=creds)
        for nombre_archivo, datos in archivos_sheets.items():
            if nombre_archivo in omitidos_por_tiempo:
                print(f"⏩ Se omite la subida de '{nombre_archivo}' porque fue omitido por tiempo.")
                continue

            ruta_archivo = os.path.join(FOLDER_PATH, nombre_archivo)
            if not os.path.exists(ruta_archivo):
                print(f"⚠️ Archivo no encontrado: {ruta_archivo}")
                continue

            with open(ruta_archivo, newline='', encoding='utf-8') as f:
                reader = csv.reader(f)
                values = list(reader)

            # Obtener la información de la hoja y el ID de la hoja
            sheet_id = datos['sheet_id']
            hoja = datos['hoja']

            # Detectar el rango de celdas que se van a actualizar (A1 hasta la última fila y columna)
            start_col = 'A'  # Empezamos en la columna A
            last_col = chr(64 + len(values[0]))  # Calculamos la última columna en base a la cantidad de columnas
            num_rows = len(values)  # Número de filas a actualizar

            # Limpiar el rango donde se pegarán los nuevos datos (de la fila 1 hacia abajo)
            clear_range = f"{hoja}!{start_col}1:{last_col}{num_rows}{max(num_rows, 10000)}"

            # Limpiar el rango de celdas sin tocar las fórmulas
            service.spreadsheets().values().clear(
                spreadsheetId=sheet_id,
                range=clear_range,
                body={}
            ).execute()
            print(f"🧹 Rango limpiado: {clear_range}")

            # Ahora, subimos los nuevos datos (encabezado + filas) a Google Sheets
            rango = f"{hoja}!A1"  # Empezamos en A1 para pegar todos los datos desde el inicio
            service.spreadsheets().values().update(
                spreadsheetId=sheet_id,
                range=rango,
                valueInputOption="RAW",
                body={"values": values}
            ).execute()
            print(f"✅ Subido '{nombre_archivo}' a hoja '{hoja}' en Google Sheets")

    except Exception as e:
        print(f"❌ Error al subir a Google Sheets: {e}")

            
            



def escribir_log(mensaje):
    ruta_log = os.path.join(FOLDER_PATH, "actualizacion.log")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(ruta_log, "a", encoding="utf-8") as log_file:
        log_file.write(f"[{timestamp}] {mensaje}\n")

# --- Funciones Google Sheets ---

def authenticate_google_sheets():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("Refrescando token de acceso...")
                creds.refresh(Request())
            except Exception as e:
                print(f"Error al refrescar token: {e}. Se requiere nueva autorización.")
                creds = None
        if not creds:
            print("Iniciando flujo de autorización...")
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
            print(f"Token guardado en {TOKEN_FILE}")

    try:
        print("Autorizando cliente gspread...")
        gc = gspread.authorize(creds)
        print("✅ Cliente gspread autorizado.")
        return gc
    except Exception as e:
        print(f"❌ Error al autorizar gspread: {e}")
        return None

def upload_csv_to_sheet(gc, spreadsheet_id, worksheet_name, csv_filepath, range_start=None , update_range=None):
    print(f"📤 Subiendo '{csv_filepath}' a '{spreadsheet_id}' -> '{worksheet_name}'")
    try:
        sh = gc.open_by_key(spreadsheet_id)
        try:
            worksheet = sh.worksheet(worksheet_name)
            print(f"   Hoja '{worksheet_name}' encontrada.")
        except gspread.WorksheetNotFound:
            print(f"   Hoja no encontrada. Creándola...")
            worksheet = sh.add_worksheet(title=worksheet_name, rows="100", cols="20")

        try:
            try:
                df = pd.read_csv(csv_filepath, sep=None, encoding='latin1', engine='python')
            except pd.errors.ParserError:
                print("⚠️ Error con ',', probando con ';'")
                df = pd.read_csv(csv_filepath, sep=';', encoding='utf-8')

            df = df.fillna('').astype(str)
            data_to_upload = [df.columns.values.tolist()] + df.values.tolist()
        except Exception as e:
            print(f"❌ Error leyendo CSV con pandas: {e}")
            return False

        if not data_to_upload or len(data_to_upload) <= 1:
            print(f"⚠️ El archivo '{csv_filepath}' está vacío o sin datos.")
            return False

        # Determinar el rango a limpiar y actualizar
        num_cols = len(data_to_upload[0])
        num_rows = len(data_to_upload)

        if worksheet.title.strip().lower() == "login time":
            # Proteger columnas A:D
            start_col_index = 5  # Columna E
            last_col_index = start_col_index + num_cols - 1
            start_col_letter = chr(64 + start_col_index)
            last_col_letter = chr(64 + last_col_index)
            clear_range = f"{start_col_letter}1:{last_col_letter}{max(num_rows, 900000)}"
            update_range = f"{start_col_letter}1"
        else:
            # Limpiar desde A1
            start_col_letter = 'A'
            last_col_letter = chr(64 + num_cols)
            clear_range = f"{start_col_letter}1:{last_col_letter}{max(num_rows, 900000)}"
            update_range = "A1"

        print(f"Rango a limpiar (relativo a hoja): {clear_range}")
        print(f"Rango para actualizar datos (relativo a hoja): {update_range}")

        worksheet.batch_clear([clear_range])
        print(f"🧹 Rango limpiado: {clear_range}")

        worksheet.update(values=data_to_upload, range_name=update_range, value_input_option='USER_ENTERED')
        print("✅ Subida completada.")
        escribir_log(f"✅ Subida exitosa: {os.path.basename(csv_filepath)} → {spreadsheet_id} / {worksheet_name}")
        return True

    except gspread.exceptions.APIError as e:
        print(f"❌ Error de API: {e}")
        if 'quota' in str(e).lower():
            print("   Posible límite alcanzado. Intenta luego.")
            escribir_log(f"❌ Error de API al subir {os.path.basename(csv_filepath)} → {spreadsheet_id} / {worksheet_name}: {e}")
        return False
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        escribir_log(f"❌ Error inesperado al subir {os.path.basename(csv_filepath)} → {spreadsheet_id} / {worksheet_name}: {e}")
        return False

def procesar_un_asunto(mail, asunto_actual):
    print(f"\n--- Procesando Asunto: '{asunto_actual}' ---")
    archivo_guardado = None
    try:
        status, data = mail.search(None, f'(SUBJECT "{asunto_actual}")')
        if status != 'OK':
            print(f"❌ Error buscando correos: {asunto_actual}")
            return None

        email_ids = data[0].split()
        print(f"🔍 Correos encontrados: {len(email_ids)}")
        if not email_ids:
            return None

        correos_con_fecha = []
        for eid in email_ids:
            status, data = mail.fetch(eid, '(RFC822)')
            if status != 'OK': continue
            raw_email = data[0][1]
            mensaje = email.message_from_bytes(raw_email, policy=policy.default)
            fecha_str = mensaje['date']
            if fecha_str:
                try:
                    fecha = parsedate_to_datetime(fecha_str)
                    if fecha.tzinfo is None:
                        fecha = pytz.utc.localize(fecha)
                    correos_con_fecha.append((fecha, mensaje))
                except Exception as e:
                    print(f"⚠️ No se pudo parsear fecha: {e}")
            else:
                print(f"⚠️ Correo sin fecha.")

        if not correos_con_fecha:
            return None

        correos_con_fecha.sort(reverse=True, key=lambda x: x[0])
        fecha_mas_reciente, mensaje = correos_con_fecha[0]

        fecha_colombia = fecha_mas_reciente.astimezone(pytz.timezone('America/Bogota'))
        print(f"📧 Más reciente: {mensaje['subject']} ({fecha_colombia.strftime('%Y-%m-%d %H:%M:%S')})")

        for parte in mensaje.iter_attachments():
            content_type = parte.get_content_type()
            nombre_archivo_original = parte.get_filename()

            if 'csv' in content_type.lower() and nombre_archivo_original:
                contenido_bytes = parte.get_payload(decode=True)
                nombre_base_seguro = ASUNTOS_A_BUSCAR.get(asunto_actual)
                if not nombre_base_seguro:
                    nombre_base_seguro = "".join(c for c in asunto_actual if c.isalnum() or c in (' ', '_', '-')).rstrip().replace(' ', '_')
                
                # Forzar nombre sin fecha ni caracteres extraños
                nombre_archivo_guardar = f"{nombre_base_seguro}.csv"
                
                ruta_completa = os.path.join(FOLDER_PATH, nombre_archivo_guardar)
                with open(ruta_completa, 'wb') as f:
                    f.write(contenido_bytes)
                print(f"✅ Archivo CSV descargado: {ruta_completa}")
                return ruta_completa

        print(f"⚠️ No se encontró CSV adjunto.")
    except Exception as e:
        print(f"❌ Error procesando asunto '{asunto_actual}': {e}")

    return archivo_guardado

def descargar_y_subir_adjuntos():
    archivos_subidos = {}
    mail = None
    gc = authenticate_google_sheets()
    if not gc:
        print("❌ No se pudo autenticar. Abortando.")
        return {}

    try:
        print("\n🔌 Conectando a Gmail...")
        mail = imaplib.IMAP4_SSL('imap.gmail.com')
        mail.login(EMAIL, PASSWORD_EMAIL)
        print("✅ Login exitoso.")
        mail.select('inbox')

        zona_horaria = pytz.timezone('America/Bogota')
        hora_actual = datetime.now(zona_horaria).hour
        fecha_actual = datetime.now(zona_horaria).strftime('%Y-%m-%d')

        # Verificar si la hora actual coincide con alguna restricción horaria
        for asunto in ASUNTOS_A_BUSCAR:
            # Verificar restricción horaria para este asunto
            if asunto in RESTRICCIONES_HORARIAS:
                if hora_actual >= RESTRICCIONES_HORARIAS[asunto]:
                    mensaje = f"⏱️ Proceso detenido por restricción horaria: {asunto} - Hora actual: {hora_actual}:00\n"
                    print(mensaje)
                    nombre_seguro = ASUNTOS_A_BUSCAR.get(asunto, asunto).replace(" ", "_")
                    with open(os.path.join(LOG_FOLDER, f"{nombre_seguro}__detenido__{fecha_actual}.txt"), 'a', encoding='utf-8') as f:
                        f.write(mensaje)
                    archivos_subidos[asunto] = "Proceso detenido por restricción horaria"
                    continue  # AHORA sí detiene el procesamiento del asunto
            if asunto in DESCARGAR_DESPUES:
                if hora_actual <= DESCARGAR_DESPUES[asunto]:
                    # Este bloque solo se ejecutará si son las 5:00 PM o después
                    mensaje = f"📥 Proceso especial iniciado para {asunto} a partir de las 5:00 PM - Hora actual: {hora_actual}:00\n"
                    print(mensaje)
                    nombre_seguro = ASUNTOS_A_BUSCAR.get(asunto, asunto).replace(" ", "_")
                    with open(os.path.join(LOG_FOLDER, f"{nombre_seguro}__proceso_especial__{fecha_actual}.txt"), 'a', encoding='utf-8') as f:
                        f.write(mensaje)
                    archivos_subidos[asunto] = "Proceso especial iniciado"
                    continue  # Detiene el procesamiento actual para proceder con el proceso especial después de las 5:00 PM

            nombre_archivo_csv = procesar_un_asunto(mail, asunto)
            if nombre_archivo_csv and os.path.exists(nombre_archivo_csv):
                if asunto in SHEET_MAPPINGS:
                    mapping = SHEET_MAPPINGS[asunto]
                    spreadsheet_id = mapping['spreadsheet_id']
                    worksheet_name = mapping['worksheet_name']
                    range_start = mapping.get('range')
                    subido_ok = upload_csv_to_sheet(gc, spreadsheet_id, worksheet_name, nombre_archivo_csv, range_start)
                    archivos_subidos[asunto] = f"{spreadsheet_id} -> {worksheet_name}" if subido_ok else "DATOS VACIOS"
                else:
                    print(f"⚠️ No hay mapeo definido para: '{asunto}'")
                    archivos_subidos[asunto] = f"Descargado localmente: {nombre_archivo_csv}"
            time.sleep(2)

    except imaplib.IMAP4.error as e:
        print(f"❌ Error IMAP: {e}")
    except Exception as e:
        print(f"❌ Error general: {e}")
    finally:
        if mail and mail.state == 'SELECTED':
            try:
                mail.close()
                mail.logout()
                print("✅ Sesión Gmail cerrada.")
            except Exception as e_logout:
                print(f"⚠️ Error cerrando sesión: {e_logout}")
        elif mail and mail.state == 'AUTH':
            try:
                mail.logout()
                print("✅ Sesión Gmail cerrada.")
            except Exception as e_logout:
                print(f"⚠️ Error cerrando sesión: {e_logout}")

    print("\n--- Resumen ---")
    if archivos_subidos:
        for asunto, resultado in archivos_subidos.items():
            print(f"  -> '{asunto}': {resultado}")
    else:
        print("  No se procesó ningún archivo.")
    return archivos_subidos
print("\n--- Proceso Finalizado ---")

if __name__ == "__main__":
    # Llamada a la función principal para procesar los correos y subir los archivos
    descargar_reportes_salesforce()
    descargar_y_subir_adjuntos()



--- Proceso Finalizado ---
Autorizando cliente gspread...
✅ Cliente gspread autorizado.

🔌 Conectando a Gmail...
✅ Login exitoso.

--- Procesando Asunto: 'Report results (Lead OE Adults' ---
🔍 Correos encontrados: 20
📧 Más reciente: Report results (Lead OE Adults 20:00) (2025-05-11 20:00:07)
✅ Archivo CSV descargado: Descargas\Lead OE Adults.csv
📤 Subiendo 'Descargas\Lead OE Adults.csv' a '1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4' -> 'Leads'
   Hoja 'Leads' encontrada.
Rango a limpiar (relativo a hoja): A1:K900000
Rango para actualizar datos (relativo a hoja): A1
🧹 Rango limpiado: A1:K900000
✅ Subida completada.

--- Procesando Asunto: 'Scheduled Report: Dials Control Panel Adults' ---
🔍 Correos encontrados: 44
📧 Más reciente: Scheduled Report: Dials Control Panel Adults (2025-05-11 20:00:19)
✅ Archivo CSV descargado: Descargas\Dials OE Adults.csv
📤 Subiendo 'Descargas\Dials OE Adults.csv' a '1KNwAl2jEjp68n2IWioi_dxYDLJY28s53YeTUkHwLcw4' -> 'Dials'
   Hoja 'Dials' encontrada.
Rang

In [ ]:
import os
import pandas as pd
import gspread
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from gspread_dataframe import set_with_dataframe

# === CONFIGURACIÓN ===
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CREDENTIALS_FILE = 'panels-id.json'
TOKEN_FILE = 'token.json'
FOLDER_PATH = 'Descargas'

archivos_sheets = {
    "Lead OE Venezuela.csv": {
        "sheet_id": "1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80",
        "hoja": "Leads"
    },
    # Agrega más archivos aquí si lo necesitas
}

# === AUTENTICACIÓN (token.json como JSON, no pickle) ===
creds = None
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        # Guarda como JSON
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

gc = gspread.authorize(creds)

# === PROCESAR ARCHIVOS CSV Y SUBIR A GOOGLE SHEETS ===
for archivo, info in archivos_sheets.items():
    try:
        ruta = os.path.join(FOLDER_PATH, archivo)
        print(f"\n📥 Procesando archivo: {archivo}")

        # Detectar delimitador
        with open(ruta, 'r', encoding='latin1') as f:
            linea = f.readline()
            delimitador = ';' if linea.count(';') > linea.count(',') else ','
        print(f"🔍 Delimitador detectado: '{delimitador}'")

        # Leer CSV
        df = pd.read_csv(ruta, delimiter=delimitador, encoding='latin1')
        df = df.fillna('').astype(str)
        print(f"📊 Registros encontrados: {len(df)}")

        # Abrir documento y hoja
        sh = gc.open_by_key(info["sheet_id"])
        worksheet = sh.worksheet(info["hoja"])
        worksheet.clear()
        set_with_dataframe(worksheet, df)

        print(f"✅ '{archivo}' subido a hoja '{info['hoja']}' en el documento {info['sheet_id']}")

    except Exception as e:
        print(f"❌ Error al subir '{archivo}': {type(e).__name__} - {e}")



📥 Procesando archivo: Lead OE Venezuela.csv
🔍 Delimitador detectado: ','
📊 Registros encontrados: 18
✅ 'Lead OE Venezuela.csv' subido a hoja 'Leads' en el documento 1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80


In [11]:
import os
import pandas as pd
import gspread
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from gspread_dataframe import set_with_dataframe

# === CONFIGURACIÓN ===
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CREDENTIALS_FILE = 'panels-id.json'
TOKEN_FILE = 'token.json'
FOLDER_PATH = 'Descargas'

archivos_sheets = {
    "Lead OE Venezuela.csv": {
        "sheet_id": "1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80",
        "hoja": "Leads"
    },
    # Agrega más archivos aquí si lo necesitas
}

# === AUTENTICACIÓN (token.json como JSON, no pickle) ===
creds = None
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        # Guarda como JSON
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

gc = gspread.authorize(creds)

# === PROCESAR ARCHIVOS CSV Y SUBIR A GOOGLE SHEETS ===
for archivo, info in archivos_sheets.items():
    try:
        ruta = os.path.join(FOLDER_PATH, archivo)
        print(f"\n📥 Procesando archivo: {archivo}")

        # Detectar delimitador
        with open(ruta, 'r', encoding='latin1') as f:
            linea = f.readline()
            delimitador = ';' if linea.count(';') > linea.count(',') else ','
        print(f"🔍 Delimitador detectado: '{delimitador}'")

        # Leer CSV
        df = pd.read_csv(ruta, delimiter=delimitador, encoding='latin1')
        df = df.fillna('').astype(str)
        print(f"📊 Registros encontrados: {len(df)}")

        # Abrir documento y hoja
        sh = gc.open_by_key(info["sheet_id"])
        worksheet = sh.worksheet(info["hoja"])
        worksheet.clear()
        set_with_dataframe(worksheet, df)

        print(f"✅ '{archivo}' subido a hoja '{info['hoja']}' en el documento {info['sheet_id']}")

    except Exception as e:
        print(f"❌ Error al subir '{archivo}': {type(e).__name__} - {e}")



📥 Procesando archivo: Lead OE Venezuela.csv
🔍 Delimitador detectado: ','
📊 Registros encontrados: 18
✅ 'Lead OE Venezuela.csv' subido a hoja 'Leads' en el documento 1i3VWFRwGhKXeNeGFEf3Wxo37eZXQg_mduwbOqySnl80
